In [1]:
import cv2  # OpenCV for image processing
import numpy as np  # NumPy for numerical operations
import matplotlib.pyplot as plt  # Matplotlib for visualization
import mss  # MSS for screen capturing
import uuid  # UUID for generating unique identifiers
import mss.tools  # Tools from MSS for image processing
import time  # Time for handling delays
import keyboard  # Keyboard for monitoring key presses
import pyautogui  # PyAutoGUI for controlling the mouse and keyboard
import math  # Math for mathematical operations

In [14]:
# actual test
def screenshot():
    with mss.mss() as sct:
        time.sleep(1)
        monitor_info = sct.monitors[0]
        top_margin = (monitor_info["height"] // 3) + 50
        monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-105}
        output = f"testimages/{str(uuid.uuid4())}.png".format(**monitor)
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    return output
            
def drive():
    img_path = screenshot()
    img_orig = cv2.imread(img_path)
    img_blur = cv2.blur(img_orig, ksize=(5, 5))
    img_canny = cv2.Canny(img_blur, threshold1=121.6, threshold2=180.9)
    cdst = cv2.cvtColor(img_canny, cv2.COLOR_GRAY2RGB)
    linesP = cv2.HoughLinesP(img_canny, 1, np.pi / 250, 90, None, 94.6, 20.5)
    
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv2.line(cdst, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)
    
    image_width = 1800
    threshold = image_width // 2  # Half of the image width

    # Separate left and right coordinates based on the threshold
    left_y = []
    left_x = []
    right_y = []
    right_x = []
    if linesP is not None:
        for line in linesP:
            x1, y1, x2, y2 = line[0]
            # Separate coordinates into left and right sides
            if x1 < threshold and x2 < threshold:
                left_y.extend([y1, y2])
                left_x.extend([x1, x2])
            elif x1 >= threshold and x2 >= threshold:
                right_y.extend([y1, y2])
                right_x.extend([x1, x2])

    # Find the lowest Y coordinates on the left and right sides
    if left_y:
        left_lowest_index = left_y.index(min(left_y))
        x1_left_lowest, y1_left_lowest = left_x[left_lowest_index], left_y[left_lowest_index]
        cv2.circle(cdst, (x1_left_lowest, y1_left_lowest), 10, (0, 0, 255), -1)  # Mark left lowest point in red

    if right_y:
        right_lowest_index = right_y.index(min(right_y))
        x2_right_lowest, y2_right_lowest = right_x[right_lowest_index], right_y[right_lowest_index]
        cv2.circle(cdst, (x2_right_lowest, y2_right_lowest), 10, (0, 0, 255), -1)  # Mark right lowest point in red


    if right_y and not left_y:
        label = 'left'
        print("right line only")
        return label
    elif left_y and not right_y:
        label = 'right'
        print("left line only")
        return label
    elif not left_y and not right_x:
        label = 'back'
        return label
    elif right_y and left_y:
        # Calculate Euclidean distance
        distance = math.sqrt((x2_right_lowest - x1_left_lowest)**2 + (y2_right_lowest - y1_left_lowest)**2)

        print("Distance between the points:", distance)

        center = distance / 2
        print('center:', center)

        desired_x_point = x1_left_lowest + center
        mario_x_point = 900

        print('Desired x point:', desired_x_point)
        print('direction: ', desired_x_point - mario_x_point)
        
        direction = desired_x_point - mario_x_point
        if(direction > 0 and direction < 110):
            label = 'forward'
        if(direction < 0 ):
            label = 'left'
        if(direction > 110):
            label = 'right'
        

        print(label)
        return label
   

def move():
    label = drive()
    print('\n')
    if (label == 'right'):
        keyboard.press('d')
        pyautogui.mouseDown(button='left')
        time.sleep(0.3)
        pyautogui.mouseUp(button='left')
        keyboard.release('d')
    if (label == 'left'):
        keyboard.press('q')
        pyautogui.mouseDown(button='left')
        time.sleep(0.3)
        pyautogui.mouseUp(button='left')
        keyboard.release('q')
    if(label == 'forward'):
        pyautogui.mouseDown(button='left')
        time.sleep(1)
        pyautogui.mouseUp(button='left')
    if(label == 'back'):
        pyautogui.mouseDown(button='right')
        time.sleep(0.4)
        pyautogui.mouseUp(button='right')

time.sleep(5)
while not keyboard.is_pressed("space"):
    move()

Distance between the points: 800.0006249997558
center: 400.0003124998779
Desired x point: 868.000312499878
direction:  -31.999687500122036
left


Distance between the points: 858.3641418419109
center: 429.1820709209554
Desired x point: 772.1820709209554
direction:  -127.81792907904457
left


Distance between the points: 875.128561983895
center: 437.5642809919475
Desired x point: 614.5642809919475
direction:  -285.4357190080525
left


Distance between the points: 889.8471778906758
center: 444.9235889453379
Desired x point: 1002.9235889453379
direction:  102.92358894533788
forward


Distance between the points: 1300.6448400697248
center: 650.3224200348624
Desired x point: 1213.3224200348623
direction:  313.32242003486226
right


left line only


Distance between the points: 605.558419972838
center: 302.779209986419
Desired x point: 605.779209986419
direction:  -294.220790013581
left


Distance between the points: 823.5593481953805
center: 411.77967409769025
Desired x point: 670.779674097